In [1]:
import numpy as np
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.metrics import pairwise
import time
from sklearn import preprocessing
import random
from numba import njit
from numba.typed import List

In [2]:
def check_pairwise_arrays(X, Y):
    if X.shape[1] != Y.shape[1]:
        raise ValueError("Incompatible dimension for X and Y matrices: "
            "X.shape[1] == %d while Y.shape[1] == %d" % (X.shape[1], Y.shape[1]))
    return X, Y

def combolutionMatrix(X, Y, k):
    m = X.shape[0]
    n = Y.shape[0]
    K = np.empty((m, n), dtype=float)
    for i in range(m):
        for j in range(n):
            K[i, j] = k(X[i],Y[j])
    return K

def updateMatrix(X, Y, k, K, support):
    for i in support:
        for j in support:
            K[i, j] = k(X[i],Y[j])
    return K

@njit
def combolutionMatrixPoly(X, Y, degree, gamma, coef0):
    m = X.shape[0]
    n = Y.shape[0]
    K = np.empty((m, n))
    C = np.empty((m, n))
    for i in range(m):
        for j in range(n):
            C[i, j] = np.dot(X[i], X[j])
            K[i, j] = (C[i, j] * gamma + coef0) ** degree
    return K, C

@njit
def updateMatrixPoly(X, K, C, t, support, degree, gamma, coef0):
    for i in support:
        for j in support:
            K[i, j] = ((C[i,j] - X[i,t]*X[j,t]) * gamma + coef0) ** degree
    return K

def polynomial_kernel(X, Y, degree=3, gamma=None, coef0=1, K=None, support=None, Cache=None):
    X, Y = check_pairwise_arrays(X, Y)

    if gamma is None:
        gamma = 1.0 / X.shape[1]
    
    if support is None:
        return combolutionMatrixPoly(X, Y, degree, gamma, coef0)
    else:
        return updateMatrixPoly(X, K, Cache, support, degree, gamma, coef0), Cache

@njit
def combolutionMatrixGauss(X, Y, gamma):
    m = X.shape[0]
    n = Y.shape[0]
    K = np.empty((m, n))
    C = np.empty((m, n))
    for i in range(m):
        for j in range(n):
            c = X[i] - X[j]
            C[i, j] = np.dot(c, c)
            K[i, j] = np.exp(-gamma * C[i, j])
    return K, C

@njit
def updateMatrixGauss(X, K, C, t, support, gamma):
    for i in support:
        for j in support:
            c = X[i,t] - X[j,t]
            K[i, j] = np.exp(-gamma * (C[i,j] - c*c))
    return K

def rbf_kernel(X, Y, gamma=None, K=None, support=None, Cache=None):
    X, Y = check_pairwise_arrays(X, Y)

    if gamma is None:
        gamma = 1.0 / X.shape[1]
    
    if support is None:
        return combolutionMatrixGauss(X, Y, gamma)
    else:
        return updateMatrixGauss(X, K, Cache, support, gamma), Cache

In [14]:
class Problem():
    def __init__(self, C=1, kernel='linear', gamma = 1.0, degree = 3):
        self.kernel = kernel
        self.C = C
        self.gamma = gamma
        self.degree = degree

        self.nY = None
        self._H_y = None
        self._H_K = None
        self._H_C = None

        self.output = False

    def computeKernelMatrix1(self, X):
        if self.kernel == 'poly':
            self._H_K = pairwise.polynomial_kernel(X, X, coef0=self.C, degree=self.degree)
            self._H_C = None
        if self.kernel == 'rbf':
            self._H_K = pairwise.rbf_kernel(X, X, gamma=self.gamma)
            self._H_C = None
        return self._H_K, self._H_C

    def computeKernelMatrix2(self, X):
        if self.kernel == 'poly': 
            K, C = polynomial_kernel(X, X, coef0=self.C, degree=self.degree)
            self._H_K = K
            self._H_C = C
        if self.kernel == 'rbf':
            K, C = rbf_kernel(X, X, gamma=self.gamma)
            self._H_K = K
            self._H_C = C
        return self._H_K, self._H_C

    def updateKernelMatrix1(self, X, i, support = None):
        flist = list(range(0, X.shape[1]))
        X_i = X[:, np.delete(flist, i)]
        if self.kernel == 'poly':
            return pairwise.polynomial_kernel(X_i, X_i, coef0=self.C, degree=self.degree), None
        if self.kernel == 'rbf':
            return pairwise.rbf_kernel(X_i, X_i, gamma=self.gamma), None

    def updateKernelMatrix2(self, X, i, support = None):
        if self.kernel == 'poly':
            gamma = 1.0 / (X.shape[1] - 1)
            return updateMatrixPoly(X, self._H_K, self._H_C, i, support, self.degree, gamma, self.C), None
        if self.kernel == 'rbf':
            return updateMatrixGauss(X, self._H_K, self._H_C, i, support, self.gamma), None

    def computeHessianMatrix(self, K):
        return np.multiply(self._H_y, K)

    def time(self, name, f):
        start_time = time.time()
        K, C = f()
        if not C is None: self._H_K = K
        if not C is None: self._H_C = C
        elapsed_time = time.time() - start_time
        if self.output:
            print(self.kernel, name, ' IN ', elapsed_time, 's:\n', K)
        else:
            print(self.kernel, name, ' IN ', elapsed_time, 's')

    def pkern(self, X, support=None):
        #print('X', X)
        self.time('SKLEARN\n', lambda: self.computeKernelMatrix1(X))
        self.time('MINE_KERN\n', lambda: self.computeKernelMatrix2(X))

        #print('X', X)
        #i = random.randint(0, X.shape[1])
        i = 0
        self.time('SKLEARN\n', lambda: self.updateKernelMatrix1(X, i, support))
        self.time('MINE_KERN\n', lambda: self.updateKernelMatrix2(X, i, support))
        
        

In [15]:
X, y = make_classification(n_samples = 4000, n_features=200, n_redundant=0)
prob = Problem(kernel='rbf')
prob.pkern(X, support=List(random.sample(range(0, X.shape[0]), int(X.shape[0]/2))))
#prob.pkern(X, support=List([0,1,2,3]))

rbf SKLEARN
  IN  0.27911376953125 s
rbf MINE_KERN
  IN  3.475484848022461 s
rbf SKLEARN
  IN  0.2875046730041504 s
rbf MINE_KERN
  IN  0.07335138320922852 s


In [5]:
m = X.shape[0]
n = X.shape[0]
K = np.zeros((X.shape[0], X.shape[0]))
C = np.zeros((X.shape[0], X.shape[0]))
for i in range(m):
    for j in range(n):
        C[i, j] = np.dot(X[i], X[j])

X_i = X[:, 0]
Ci = np.zeros((X.shape[0], X.shape[0]))
Cx = Ci
for i in range(m):
    for j in range(n):
        Ci[i, j] = np.dot(X_i[i], X_i[j])
        Cx[i, j] = C[i,j] - X[i,0]*X[j,0]

print(X)
print()
print(C)
print()
print(Ci)
print()
print(Cx)

KeyboardInterrupt: 